In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
main_dir = '/content/drive/MyDrive/Colab Notebooks/NTC'
checkpoint_py = '/content/drive/MyDrive/Colab Notebooks/FL/checkpoint_manager.py'
utils_py = '/content/drive/MyDrive/Colab Notebooks/FL/utils.py'

In [ ]:
%cd $main_dir
!ls

/content/drive/MyDrive/Colab Notebooks/NTC
Accuracy.png		Flower_FL  __pycache__	     utils.py
Accuracy_Zoom.png	Loss.png   Result.ipynb
checkpoint_manager.py	models	   sdn_results
Federated_Learning_NTC	NTC.ipynb  sdn_saved_models


In [ ]:
# !cp "$checkpoint_py" .
# !cp "$utils_py" .

In [ ]:
!python --version
!pip --version

Python 3.10.12
pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)


In [ ]:
!pip install --quiet --upgrade tensorflow-federated
!pip install -U tensorboard_plugin_profile

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.3/68.3 MB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.5/103.5 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 kB 9.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 558.5/558.5 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 MB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 26.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.8/255.8 kB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 238.9/238.9 kB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 359.4/359.4 kB 37.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
%load_ext tensorboard

In [ ]:
# from checkpoint_manager import FileCheckpointManager
from pathlib import Path
import nest_asyncio
import matplotlib.pyplot as plt
import sys
import pandas as pd
import tensorflow as tf
# import tensorflow_federated as tff
# from utils import plot_graph
import numpy as np
from time import time
import os
import logging
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import classification_report

In [ ]:
logging.disable(logging.WARNING)
np.set_printoptions(threshold=sys.maxsize)
nest_asyncio.apply()
SEED = 1337
tf.random.set_seed(SEED)

In [ ]:
gpu_devices = tf.config.list_physical_devices('GPU')
if not gpu_devices:
  raise ValueError('Cannot detect physical GPU device in TF')
tf.config.list_logical_devices()

[LogicalDevice(name='/device:CPU:0', device_type='CPU'),
 LogicalDevice(name='/device:GPU:0', device_type='GPU')]

### **Helper Function**

In [ ]:
def train_test_split(df, frac=0.2):
    selected = df['flow_id'].drop_duplicates().sample(frac=frac)
    test = df[df['flow_id'].isin(selected)]
    train = df[~df['flow_id'].isin(selected)]
    return train, test

In [ ]:
def test_case_split(df, split):
    # return 1 dict chia du lieu cho cac may
    selected = df['flow_id'].drop_duplicates().sample(frac=1)
    # print(selected.shape)
    total_data_count = selected.shape[0]
    data_per_set = int(np.floor(total_data_count/split))
    DataFrameDict = {}
    for i in range(1, split+1):
        client_name = "client_" + str(i)
        start = data_per_set * (i-1)
        end = data_per_set * i

        print(f"Adding data from {start} to {end} for client : {client_name}")
        DataFrameDict[client_name] = df[df['flow_id'].isin(
            selected[start:end])]
    return DataFrameDict

In [ ]:
def sec_to_hours(seconds):
    a = seconds//3600
    b = (seconds % 3600)//60
    c = (seconds % 3600) % 60
    d = "{:.0f} hours {:.0f} mins {:.0f} seconds".format(a, b, c)
    return d

In [ ]:
def most_frequent(List):
    return max(set(List), key=List.count)

Thay đổi các siêu tham số trong cell dưới <br>
**experiment_name:** Tên bộ dữ liệu được sử dụng, <Tên dữ liệu>_<Số byte sử dụng> <br>
**method:** Mô hình sử dụng <br>
**client_lr, server_lr:** Learning rate của client và server, khi fine tune bắt đầu từ 1 và giảm dần xuống 3e-4 <br>
**NUM_ROUNDS:** Số vòng lặp (Bắt đầu từ 1 và tăng dần lên 3000 nếu mô hình chưa hội tụ - 1/100/200/300/400/500/.../3000) <Br>
**BATCH_SIZE:** Kích thước batch (Bắt đầu từ 8 và tăng dần lên 64) 8/16/32/64 <br>
**split:** Để nguyên là 5
<br>
**byte_number:** (string): Cac byte cua packet, bao gom 10, 32, 64, 128, 256, 512, 1024, 1460

In [ ]:
from models.CNN_2D import create_keras_model

In [ ]:
byte_number = "128"
experiment_name = "GQUIC_small_" + byte_number
method = "FL_CNN_2D"

client_lr = 0.01
server_lr = 1
NUM_ROUNDS = 10
BATCH_SIZE = 64
PACKET_NUM = 20
split = 5

In [ ]:
# this_dir = Path.cwd()
# model_dir = this_dir / "sdn_saved_models" / experiment_name / method
# output_dir = this_dir / "sdn_results" / experiment_name / method

# if not model_dir.exists():
#     model_dir.mkdir(parents=True)

# if not output_dir.exists():
#     output_dir.mkdir(parents=True)

### **Đọc dữ liệu**

In [ ]:
train_dir = '/content/drive/MyDrive/Colab Notebooks/FL/GQUIC_small/Train/GQUIC_train_' + byte_number + '.feather'
test_dir = '/content/drive/MyDrive/Colab Notebooks/FL/GQUIC_small/Test/GQUIC_test_' + byte_number + '.feather'
data = pd.read_feather(train_dir)
test = pd.read_feather(test_dir)

In [ ]:
result = test.groupby('flow_id')['Label'].apply(list).to_dict()
flow_label = []
for flow in result:
    flow_label.append(most_frequent(result[flow]))

flow_label = np.array(flow_label)
true_test = test.drop('flow_id', axis=1)
NUM_FEATURE = len(true_test.columns)-1
NUM_CLASSES = len(np.unique(true_test['Label']))
print(NUM_FEATURE, NUM_CLASSES)

128 5


# **Federated Learning Approach**

In [ ]:
def make_tf_dataset(dataframe, batch_size=None):
    result = dataframe.groupby('flow_id')['Label'].apply(list).to_dict()
    y = []
    for flow in result:
        y.append(most_frequent(result[flow]))
    y = np.array(y)
    y = y.reshape(-1, 1)
    dataframe = dataframe.drop(['Label', 'flow_id'], axis=1).to_numpy()/255
    dataframe = dataframe.reshape(-1, PACKET_NUM, NUM_FEATURE)
    dataframe = np.expand_dims(dataframe, -1)

    dataset = tf.data.Dataset.from_tensor_slices((dataframe, y))
    # kieu du lieu tensorflow cho fd
    if batch_size:
        dataset = dataset.batch(batch_size)
    return dataset

In [ ]:
DataFrameDict = test_case_split(data, split)

Adding data from 0 to 2633 for client : client_1
Adding data from 2633 to 5266 for client : client_2
Adding data from 5266 to 7899 for client : client_3
Adding data from 7899 to 10532 for client : client_4
Adding data from 10532 to 13165 for client : client_5


In [ ]:
train_data, val_data = [], []
for client_data in DataFrameDict.keys():
    train_df, val_df = train_test_split(DataFrameDict[client_data], frac=0.2)
    # TF Datasets
    train_data.append(make_tf_dataset(train_df, batch_size=BATCH_SIZE))
    val_data.append(make_tf_dataset(val_df, batch_size=1))

## **Model Definition**

In [ ]:
def input_spec():
    return (
        tf.TensorSpec([None, PACKET_NUM, NUM_FEATURE, 1], tf.float64),
        tf.TensorSpec([None, 1], tf.int64)
    )

In [ ]:
def model_fn():
    model = create_keras_model(NUM_FEATURE, NUM_CLASSES)

    return tff.learning.models.from_keras_model(
        model,
        input_spec=input_spec(),
        loss=tf.keras.losses.SparseCategoricalCrossentropy(),
        metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

## **Train Model**

Each time the **next** method is called, the server model is broadcast to each client using a broadcast function. For each client, **one** epoch of local training is performed. Each client computes the difference between the client model after training and the initial broadcast model. These model deltas are then aggregated at the server using some aggregation function.

In [ ]:
tff_train_acc = []
tff_train_loss = []
tff_val_acc = []
tff_val_loss = []

In [ ]:
iterative_process = tff.learning.algorithms.build_weighted_fed_avg(
    model_fn,
    client_optimizer_fn=lambda: tf.keras.optimizers.Adam(
        learning_rate=client_lr),
    server_optimizer_fn=lambda: tf.keras.optimizers.Adam(
        learning_rate=server_lr)
)

In [ ]:
# Evaluate model
evaluation_process = tff.learning.algorithms.build_fed_eval(model_fn)
evaluation_state = evaluation_process.initialize()

In [ ]:
# Initialize model state
state = iterative_process.initialize()

In [ ]:
current_round = 0
rounds_per_eval = 2

### **Training**

In [ ]:
for i in range(current_round, NUM_ROUNDS):

    start = time()
    # Train
    result = iterative_process.next(state, train_data)
    state = result.state
    train_metrics = result.metrics['client_work']['train']
    duration = time() - start

    # Print
    print('round {:2d}\ntrain_loss={l:.3f}, train_acc={ac:.3f}, time={d:.3f} secs'.format(
        i+1, l=train_metrics['loss'], ac=train_metrics['sparse_categorical_accuracy'], d=duration))

    # Validation
    if i % rounds_per_eval == 0 or i == NUM_ROUNDS - 1:
      evaluation_state = evaluation_process.set_model_weights(evaluation_state,result.state.global_model_weights)
      evaluation_output = evaluation_process.next(evaluation_state, val_data)
      val_metrics = evaluation_output.metrics['client_work']['eval']['current_round_metrics']
      print('val_loss: {:.3f} val_acc: {:.3f}'.format(
        val_metrics['loss'], val_metrics['sparse_categorical_accuracy']))

round  1
train_loss=11.474, train_acc=0.273, time=246.817 secs
val_loss: 11.573 val_acc: 0.282
round  2
train_loss=11.528, train_acc=0.285, time=233.895 secs
round  3
train_loss=11.528, train_acc=0.285, time=236.100 secs
val_loss: 11.573 val_acc: 0.282
round  4
train_loss=11.528, train_acc=0.285, time=235.394 secs
